# Linkage mapping using Lep-MAP3

ParentCall2 module to call possible missing or erroneous genotypes

In [ ]:
java -cp ParentCall2 \
data=pedigree-subset.txt \
vcfFile=snps-10.vcf \
removeNonInformative=1 \
halfSibs=1 \
ZLimit=1 \
> data-zlimit.call


Split the ParentCall2 output file by chromosome (ran in R 4.2.1)

In [ ]:
# read in output of Parentcall2
data <- read.delim("data-zlimit.call", header = FALSE)
dataPedi <- data[1:7,]
dataGeno <- tail(data,-7)
#dataGeno$V1 <- as.numeric(dataGeno$V1)

# for each chromosome in list, create dataset and output
for(i in 1:35){
  chrSplit <- dataGeno[dataGeno$V1 %in% c(i,paste0(i,"_unplaced")),]
  chrSplit2 <- rbind(dataPedi,chrSplit)
  write.table(chrSplit2, file = paste0("data-chr", i, ".call"), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)
}


SeparateChromosomes2 module to cluster SNPs into linkage groups

In [ ]:
for i in {1..33} 35
do
    java -cp SeparateChromosomes2 \
    data=data-chr$i.call \
    lodLimit=5 theta=0.03 sizeLimit=1 numThreads=32 \
    > mapChr$i-5.03.1.txt
    sort mapChr$i-5.03.1.txt|uniq -c|sort -n > mapChr$i-5.03.1-sort.txt
    continue
done

grep . *-sort.txt > mapChr-sort-all.txt


Retain SNPs assigned to the largest linkage group of each chromosome and merge SNPs from all chromosomes into a single file (ran in R 4.2.1)

In [ ]:
library(plyr)
library(dplyr)

# loop
ilist <- 1:35
ilist2 <- ilist[-34]
for(i in ilist2){
  # read in map file of each chromosome
  data <- read.delim(paste0("mapChr", i, "-5.03.1.txt"), header = TRUE)
  colnames(data) <- "lg"
  # manipulate linkage group assignments
  data$lg[data$lg != "1"] <- "0" # for all linkage group assignments that is not '1', replace with '0'
  data$lg[data$lg == "1"] <- i # for all linkage groups assignments that is '1', replace with chromosome number
  # bind with chr and marker names
  snpInfo <- read.delim(paste0("data-chr",i,"-placed.call"), header = FALSE) # snp chr and pos information
  snpInfo2 <- snpInfo[-(1:7),]
  chrPos <- snpInfo2[,(1:2)]
  colnames(chrPos) <- c("chr","pos")
  name <- paste0("chr",i)
  assign(name, cbind(chrPos,data))
}
  
# combine all map files
chrList <- mget(paste0("chr", ilist2))
allChr <- ldply(chrList, data.frame)

# reorder lg assignment according to data.call
allCall <- read.delim("data-zlimit.call", header = FALSE) # snp chr and pos information
allCall2 <- allCall[-(1:7),]
allCall3 <- allCall2[,(1:2)]
colnames(allCall3) <- c("chr","pos")

# get new lg assignment if chr and pos matches
allCall3$lg <- allChr$lg[match(interaction(allCall3[c("chr", "pos")]), interaction(allChr[c("chr","pos")]))]

allCall3$lg[is.na(allCall3$lg)] <- 0

# output SNPs
write.table(allCall3, file = "mapChrAll-5.03.1-full.txt", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)
mapChrAll <- as.data.frame(allCall3$lg)
colnames(mapChrAll) <- "#java SeparateChromosomes2  data=data-zlimit.call lodLimit=5 theta=0.03 sizeLimit=1 numThreads=32"
write.table(mapChrAll, file = "mapChrAll-5.03.1.txt", sep = "\t", row.names = FALSE, col.names = TRUE, quote = FALSE)


OrderMarkers2 module to infer SNP order within each linkage group

In [ ]:
# a priori map (code is for each chromosome)
for i in {1..10}
do
    java -cp OrderMarkers2 \
    map=mapChrAll-5.03.1.txt \
    data=data-zlimit.call \
    numThreads=4 \
    chromosome=1 \
    scale=3M/N 3 \ # only applied to the seven largest chromosomes
    numMergeIterations=24 \
    numPolishIterations=8 \
    computeLODScores=order-1-r$i-LOD.txt \
    calculateIntervals=order-1-r$i.intervals \
    > order-1-r$i.txt
done

for i in {1..10}
do
    #get sex-averaged map
    java -cp OrderMarkers2 \
    evaluateOrder=order-1-r$i.txt \
    data=data-zlimit.call \
    improveOrder=0 \
    sexAveraged=1 \
    scale=3M/N 3 \
    > order-1-r$i-SA.txt
done

# physical map (code is to loop through all chromosomes)
for i in {1..33} 35
do
    java -cp OrderMarkers2 \
    data=data-zlimit.call \
    evaluateOrder=order-chr$i.txt \
    improveOrder=0 \
    numThreads=4 \
    > order-physical-$i.txt
done

for i in {1..33} 35
do
    #get sex-averaged map
    java -cp OrderMarkers2 \
    evaluateOrder=order-physical-$i.txt \
    data=data-zlimit.call \
    improveOrder=0 \
    sexAveraged=1 \
    > order-physical-$i-SA.txt
done


Merge male, female, and sex averaged maps

In [ ]:
## append SA value to -rX.txt file
sed -n '1,2p' order-physical-$i.txt > order-physical-$i-top-temp.txt #extract command and likelihood
tail -n +3 order-physical-$i.txt | cut -f 1-3 > order-physical-$i-left-temp.txt #extract marker, male, female
tail -n +3 order-physical-$i-SA.txt | cut -f 2 > order-physical-$i-mid-temp.txt #extract average
var="average_position"
sed -i "1s/.*/$var/" order-physical-$i-mid-temp.txt #rename first line of average
tail -n +3 order-physical-$i.txt | cut -f 4- > order-physical-$i-right-temp.txt #extract phased data
paste order-physical-$i-left-temp.txt order-physical-$i-mid-temp.txt order-physical-$i-right-temp.txt > order-physical-$i-temp.txt #cbind marker, positions, phased data
cat order-physical-$i-top-temp.txt order-physical-$i-temp.txt > order-physical-$i-merged.txt #rbind with command and likelihood
rm *-temp.txt


Applying linkage map to assemble female reference Z-linked contigs using agptools 

In [ ]:
agptools split split.txt Ncf_Z.agp > Ncf_Z-split.agp
agptools join -n 100 join.txt Ncf_Z-split.agp > Ncf_Z-join.agp
agptools remove remove.txt Ncf_Z-join.agp > Ncf_Z-final.agp


# Linkage map evaluation

Evaluate linkage maps using the LMPlot module in Lep-MAP3 which makes a Lep-MAP graph to detect mapping errors

In [ ]:
# code looks through all chromosomes (physical map)
for i in {1..35}
do
    java -cp LMPlot \
    order-physical-$i-merged.txt \
    > order-physical-$i-merged.dot
    dot -Tpng order-physical-$i-merged.dot > order-physical-$i-merged.png
done


# Linkage map summary and visualisation

Visualise linkage maps in LinkageMapView (ran in R 4.2.1)

In [ ]:
# load packages
library(LinkageMapView)
library(dplyr)

# list of chromosomes to plot
ilist <- c(1:15,17:24,26:28,30,32)
jlist <- c(1:15,17:24,26:28, "1A", "4A")

## import results
# import autosomes
mapply(function(i, j) {
  # import file
  setwd("order-physical") # folder containing outputs from OrderMarkers2
  lep <- read.delim(Sys.glob(paste0("order-physical-",i,"-*.txt")), header = FALSE)
  # manipulate file
  lep2 <- lep[-(1:3),c(1,4)]
  colnames(lep2) <- c("locus", "position")
  lep2$position <- as.numeric(lep2$position)
  lep2$group <- j
  lep2$group <- as.character(lep2$group)
  name <- paste0("data",i)
  assign(name, lep2, envir = .GlobalEnv)
  },
  i = ilist,
  j = jlist,
  SIMPLIFY = FALSE)
# import chr Z
setwd("order-physical")
lep <- read.delim(Sys.glob(paste0("order-physical-33-*.txt")), header = FALSE)
# manipulate file
data33 <- lep[-(1:3),c(1,2)]
colnames(data33) <- c("locus", "position")
data33$position <- as.numeric(data33$position)
data33$group <- "Z"
data33$group <- as.character(data33$group)

# draw tickmarks at each cM from 0 to largest position of linkage groups to be drawn
list <- c(1:15,17:24,26:28,30,32,33)

maxAll <- data.frame(matrix(NA,    # Create empty data frame
                            nrow = 1,
                            ncol = 2))
colnames(maxAll) <- c("chr", "max")

for(i in list){
  data <- get(paste0("data",i))
  maxChr <- data.frame(matrix(NA,    # Create empty data frame
                              nrow = 1,
                              ncol = 2))
  colnames(maxChr) <- c("chr", "max")
  maxChr[1,1] <- i
  maxChr[1,2] <- floor(max(data$position))
  maxAll <- rbind(maxAll,maxChr)
}

maxAll <- na.omit(maxAll)

maxpos <- floor(max(maxAll$max))
at.axis <- seq(0, maxpos)

# put labels on ruler at every 10 cM
axlab <- vector()

for (lab in 0:maxpos) {
  if (!lab %% 10) {
    axlab <- c(axlab, lab)
  }
  else {
    axlab <- c(axlab, NA)
  }
}

# concat dataframes
dataList <- mget(paste0("data",list))

dataAll <- bind_rows(dataList)
dataAll2 <- dataAll[,c(3,2,1)]

chrlist <- c(1, "1A", 2:4,"4A", 5:15,17:24,26:28,"Z")
chrlistName <- c("1 ", "1A", "2 ", "3 ", "4 ", "4A", "5 ", "6 ", "7 ", "8 ", "9 ",10:15, 17:24, 26:28, "Z ")

#palette <- c("#3E8AB9", "#4E9FB2", "#65B9A8", "#8CD1A4", "#DEF19D", "#D03A51")
#palette <- c("#274C7750", "#5C737250", "#91996C50", "#FAE66150", "#F29B4C50", "#E94F3750")
palette <- c("#007fff", "#8BD0A4", "#fed767","#F5AE70","#DC3220")

sectcoldfdf <- lmvdencolor(dataAll2, colorin = palette, bias = 1.5)

# plot
outfile = file.path("path/",
                    "chr_denmap-all.pdf")

lmv.linkage.plot(dataAll2, outfile, pdf.height = 20, pdf.width = 18, pdf.bg = "transparent",
                 mapthese = chrlist, lgw = 0.4, denmap = TRUE,
                 at.axis = at.axis, labels.axis = axlab, ylab = "Map length (cM)",
                 cex.axis = 1, cex.lgtitle = 1, lg.lwd = 3, lwd.axis = 3, 
                 sectcoldf = sectcoldfdf)
